In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [3]:
from src.slide_lfi import train_mdn, load_sims_from_prior, run_sims_from_prior
import torch
import numpy as np
from src.models.MixtureDensityNetwork import MixtureDensityNetwork
import pandas as pd

In [4]:
low = 0.1
high = 1.0
torch.manual_seed(1234)
np.random.seed(1234)

In [5]:
def uniform_prior():
    return np.random.uniform(low, high, 1)[0]

In [6]:
df, x_train, y_train = load_sims_from_prior()

FileNotFoundError: [Errno 2] No such file or directory: 'stats_100000_sims.pickle'

In [7]:
df.head(10)

,friction,mean,var
0,0.100000,0.575468,1.662869e-02
1,0.795544,0.038569,9.798389e-07
2,0.918384,0.019121,1.072794e-06
3,0.356726,0.143673,7.582076e-04
4,0.650934,0.065674,2.071131e-05
5,0.817180,0.034364,9.366320e-06
6,0.639239,0.068166,6.141881e-06
7,0.176296,0.342026,1.242401e-04
8,0.836833,0.031136,2.682287e-05
9,0.426916,0.105551,1.581049e-04


In [7]:
network = MixtureDensityNetwork([20], 2)
network.fit(x_train, y_train, batch_size=1000, epochs=30000, verbose=False)

Training mdn network on 10000 datapoints
Initial Loss is: 1.204181432723999
Training Finished, final loss is -2.5343918800354004


In [20]:
sample_means = np.array(df.head(10).loc[:, 'mean'])[:, np.newaxis]

In [21]:
all_posteriors = []
for mean in sample_means:
    x_obs = torch.from_numpy(np.float32(np.array([mean])))
    all_posteriors.append(network.get_mog(x_obs))

In [22]:
pred = []
for mog in all_posteriors:
    pred.append(mog.gen()[0])

In [23]:
x_true = np.array(df.head(10).loc[:, 'friction'])[:, np.newaxis]

In [25]:
x_pred = np.array(pred)[:, np.newaxis]

In [27]:
result_df = pd.DataFrame(np.concatenate([x_true, x_pred], axis=1), columns=["true", "pred"])

In [28]:
result_df

,true,pred
0,0.100000,0.095602
1,0.795544,0.795283
2,0.918384,0.896579
3,0.356726,0.339578
4,0.650934,0.663590
5,0.817180,0.792185
6,0.639239,0.623979
7,0.176296,0.189820
8,0.836833,0.866591
9,0.426916,0.434510
